In [1]:
!pip install hydra-core

     |████████████████████████████████| 133kB 7.7MB/s 
     |████████████████████████████████| 112kB 25.3MB/s 
     |████████████████████████████████| 645kB 22.6MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-cp37-none-any.whl size=141231 sha256=e6f0d035e600f71853065ffdbf4e80bed48811b2030e60938169518a7e44fe18
  Stored in directory: /root/.cache/pip/wheels/e3/e2/fa/b78480b448b8579ddf393bebd3f47ee23aa84c89b6a78285c8
Successfully built antlr4-python3-runtime
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import torch

In [3]:
bart = torch.hub.load('pytorch/fairseq', 'bart.large')
bart.eval()

Downloading: "https://github.com/pytorch/fairseq/archive/master.zip" to /root/.cache/torch/hub/master.zip


running build_ext
cythoning fairseq/data/data_utils_fast.pyx to fairseq/data/data_utils_fast.cpp


/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py:369: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))


cythoning fairseq/data/token_block_utils_fast.pyx to fairseq/data/token_block_utils_fast.cpp
building 'fairseq.libbleu' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/fairseq
creating build/temp.linux-x86_64-3.7/fairseq/clib
creating build/temp.linux-x86_64-3.7/fairseq/clib/libbleu
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.7m -c fairseq/clib/libbleu/libbleu.cpp -o build/temp.linux-x86_64-3.7/fairseq/clib/libbleu/libbleu.o -std=c++11 -O3 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE="_gcc" -DPYBIND11_STDLIB="_libstdcpp" -DPYBIND11_BUILD_ABI="_cxxabi1011" -DTORCH_EXTENSION_

100%|██████████| 3699866548/3699866548 [01:51<00:00, 33288783.72B/s]
1042301B [00:00, 3576745.49B/s]
456318B [00:00, 10315102.98B/s]


BARTHubInterface(
  (models): ModuleList(
    (0): BARTModel(
      (encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
        (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dropout_module): FairseqDropo

In [10]:
x = bart.fill_mask(["Gina's friend Tami had a folder that Gina wanted. And since <mask> <mask> <mask>. So she decided she would try to find the folder on her own."], topk=1, beam=10, match_source_len=False)

In [11]:
x[0]

[("Gina's friend Tami had a folder that Gina wanted. And since Tami was gone, Gina didn't know where it was. So she decided she would try to find the folder on her own.",
  tensor(-0.5008))]

In [6]:
x[0][1][0]

"Gina's friend Tami had a folder that Gina wanted. And since Tami wasn't around, Gina didn't know where it was. So she decided she would try to find the folder on her own."

In [7]:
val_lines_list = []
val_file = open('val.source')
val_lines = val_file.readlines()
for line in val_lines:
  val_lines_list.append(line)

In [8]:
val_lines_list2 = []
for line in val_lines_list:
  insert_str = "And since <mask> <mask> <mask>."
  line_blocks = line.split("#")
  if (len(line_blocks) == 2): # if there is only one #
    new_str = line_blocks[0] + insert_str + line_blocks[1]
    val_lines_list2.append(new_str)

In [9]:
len(val_lines_list2)

14313

In [ ]:
print(56/14313)

0.003912527073290016


In [14]:
target_lines = []
counter = 0
limit = 1000
temp_line = ""
for masked_line in val_lines_list2:
  if (counter < limit):
    if (temp_line == masked_line):
      counter += 1
      continue
    x = bart.fill_mask([masked_line], topk=1, beam=10, match_source_len=False)
    target_lines.append(x[0][0][0])
    print(x[0][0][0])
    counter += 1
    print(str(counter) + " / " + str(limit) + "   ---   " + str(round(counter/limit, 4)) + "%")
    print()
    print()
    temp_line = masked_line
  else:
    break

The Smiths were having photos done of the children. Ty's face lit up as he ran to the new toy, happily posing for photos. And since then, he's been playing with it ever since.
1 / 1000   ---   0.001%


Timmy was running fast to class. Timmy later found out that the exam was to be on time, he failed. And since then, Timmy has never been the same.
6 / 1000   ---   0.006%


I was in line at the cinema. The boy proved her wrong by wetting himself. And since then, I’ve never been the same.
10 / 1000   ---   0.01%


Zoey is an only child. And since she's an only child, she can't wait to meet her brother. When the baby is born, she goes to the hospital to meet her brother!
13 / 1000   ---   0.013%


Hannah sent her kids to go to the shoreline. As a result, they had to get out of the water and put on sunscreen. And since it was so hot, they didn't want to get wet.
17 / 1000   ---   0.017%


It was a cold night and the campers were tired and hungry. They went off to their tents after a few hour

In [16]:
with open('enthymeme.hypo', 'w') as f:
    for item in target_lines:
        f.write("%s\n" % item)

In [15]:
target_lines

["The Smiths were having photos done of the children. Ty's face lit up as he ran to the new toy, happily posing for photos. And since then, he's been playing with it ever since.",
 'Timmy was running fast to class. Timmy later found out that the exam was to be on time, he failed. And since then, Timmy has never been the same.',
 'I was in line at the cinema. The boy proved her wrong by wetting himself. And since then, I’ve never been the same.',
 "Zoey is an only child. And since she's an only child, she can't wait to meet her brother. When the baby is born, she goes to the hospital to meet her brother!",
 "Hannah sent her kids to go to the shoreline. As a result, they had to get out of the water and put on sunscreen. And since it was so hot, they didn't want to get wet.",
 'It was a cold night and the campers were tired and hungry. They went off to their tents after a few hours to sleep. And since then, they have never looked back.',
 'Amy was in the park walking her dog Spot. Amy gav